In [1]:
from pipeline import Pipeline
from pipeline.steps import *

xgb_params = {
    "colsample_bylevel": 0.4778015829774066,
    "colsample_bynode": 0.362764358742407,
    "colsample_bytree": 0.7107423488010493,
    "gamma": 1.7094857725240398,
    "learning_rate": 0.02213323588455387,
    "max_depth": 20,
    "max_leaves": 512,
    "min_child_weight": 16,
    "n_estimators": 1667,
    "n_jobs": -1,
    "random_state": 42,
    "reg_alpha": 39.352415706891264,
    "reg_lambda": 75.44843704068275,
    "subsample": 0.06566669853471274,
    "verbose": 0,
}

class FeatureEngineeringProductCatInteractionStep(PipelineStep):

    def __init__(self, cat="cat1", name: Optional[str] = None, tn="tn"):
        super().__init__(name)
        self.cat = cat
        self.tn = tn


    def execute(self, df, df_original=None) -> None:
        # agrupo el dataframe por cat1 (sumando), obteniendo fecha, cat1 y
        # luego paso el dataframe a wide format, donde cada columna es una categoria  y la fila es la suma de tn para cada cat1
        # luego mergeo al dataframe original por fecha y product_id
        df_index = df.index
        if df_original is None:
            df_to_proces = df
        else:
            df_to_proces = df_original
        df_cat = df_to_proces.groupby(["date_id", self.cat]).agg({self.tn: "sum"}).reset_index()
        print(df_cat)
        df_cat = df_cat.pivot(index="date_id", columns=self.cat, values=self.tn).reset_index()
        # paso a string los nombres de las columnas
        df_cat.columns = [f"{self.tn}_{self.cat}_{col}" if col != "date_id" else "date_id" for col in df_cat.columns]
        df = df.merge(df_cat, on="date_id", how="left")
        # vuelvo a setear el indice original
        df.index = df_index
        return {"df": df}
    

import numpy as np
class Log1pTranformation(PipelineStep):
    def execute(self, df):
        df["tn"] = df["tn"].apply(lambda x: np.log1p(x) if x >= 0 else 0)
        return {"df": df}
    
class InverseLog1pTranformation(PipelineStep):
    def execute(self, df, predictions, y_test):
        df["target"] = df["target"].apply(lambda x: np.expm1(x) if x >= 0 else 0)
        predictions = predictions.apply(lambda x: np.expm1(x) if x >= 0 else 0)
        y_test["target"] = y_test["target"].apply(lambda x: np.expm1(x) if x >= 0 else 0)
        return {"df": df, "predictions": predictions, "y_test": y_test}

class GroupByProductStep(PipelineStep):
    def execute(self, df) -> None:
        # Agrupo el dataframe por product_id y fecha, sumando las cantidades
        df = df.groupby(["product_id", "fecha"]).agg({
            'cust_request_qty': 'sum',
            'cust_request_tn': 'sum',
            'tn': 'sum',
            'stock_final': 'max',
            'cat1': 'first',
            'cat2': 'first',
            'cat3': 'first',
            'brand': 'first',
            'sku_size': 'max',
        }).reset_index()
        # le dejo la columna customer_id = 0 para que no crashee el resto
        df['customer_id'] = 1
        return {"df": df}

class FilterDatasetByColumn(PipelineStep):
    def __init__(self, column: str, value, name: Optional[str] = None):
        super().__init__(name)
        self.column = column
        self.value = value
        
    def execute(self, df) -> None:
        # Filtra el DataFrame por el valor de la columna especificada
        df_original = df.copy()
        df_filtered = df[df[self.column] == self.value]
        print(df_filtered.shape)
        return {"df": df_filtered, "df_original": df_original}

class DeleteBadColumns(PipelineStep):
    def execute(self, df) -> None:
        # Elimina  las columnas donde toda sus filas son NaN
        base_columns = df.columns
        df = df.dropna(axis=1, how='all')
        df = df.loc[:, (df != 0).any(axis=0)]
        deleted_columns = set(base_columns) - set(df.columns)
        print(f"Deleted columns: {deleted_columns}")
        return {"df": df}

class CreateResidualTargetStep(PipelineStep):
    def __init__(self, name: Optional[str] = None, target_col: str = 'tn', window: int = 12):
        super().__init__(name)
        self.target_col = target_col
        self.window = window

    def execute(self, df: pd.DataFrame) -> Dict:
        df = df.sort_values(['product_id', 'customer_id', 'fecha']).copy()

        # Valor futuro (shift -2) que será la predicción final
        df['target_shifted'] = df.groupby(['product_id', 'customer_id'])[self.target_col].shift(-2)

        # Cálculo del valor base usando la media rolling pasada
        df['base_prediction'] = (
            df.groupby(['product_id', 'customer_id'])[self.target_col]
            .transform(lambda x: x.rolling(self.window, min_periods=1).mean())
        )

        # Target: residuo a predecir
        df['target'] = df['target_shifted'] - df['base_prediction']
        df.drop(columns=['target_shifted', 'base_prediction'], inplace=True, errors='ignore')

        return {
            "df": df,
            "target_col": self.target_col,
        }

class InverseResidualTargetStep(PipelineStep):
    def __init__(
        self,
        name: Optional[str] = None,
        target_col: str = 'tn',
        window: int = 12
    ):
        super().__init__(name)
        self.target_col = target_col
        self.window = window

    def execute(self, df: pd.DataFrame, predictions, y_test) -> Dict:

        # Orden correcto
        df = df.sort_values(['product_id', 'customer_id', 'fecha'])

        # Recalcular base_prediction exactamente como en CreateResidualTargetStep
        df['base_prediction'] = (
            df.groupby(['product_id', 'customer_id'])[self.target_col]
            .transform(lambda x: x.rolling(self.window, min_periods=1).mean())
        )

        # Invertir el target residual: target_original = residual + base_prediction
        df['target'] = df['target'] + df['base_prediction']

        predictions = predictions + df['base_prediction'].loc[y_test.index]

        y_test = df[["target"]].loc[y_test.index]
        # Limpiar si no querés guardar la base
        df.drop(columns=['base_prediction'], inplace=True)

        return {"df": df, "predictions": predictions, "y_test": y_test}
    



In [2]:
pipeline = Pipeline(
    steps=[
        LoadDataFrameStep("df_intermedio.parquet"),
    ]
)
pipeline.run()

Executing step: LoadDataFrameStep
Step LoadDataFrameStep completed in 2.27 seconds


In [3]:
df = pipeline.get_artifact("df")
# get unique brands
brands = df["brand"].unique()
del df
import gc
gc.collect()
brands

['ARIEL', 'LIMPIEX', 'NATURA', 'DOWNY', 'ROPEX1', ..., 'INDUSTRIAL', NaN, 'TWININGS', 'SKIN1', 'VICHY']
Length: 38
Categories (37, object): ['ARIEL', 'AYUDIN', 'CAPILATIS', 'COLBERT', ..., 'SKIN1', 'TWININGS', 'VICHY', 'VIVERE']

In [4]:
pipeline_results = {}
for brand in brands:
    print(f"Processing brand: {brand}")
    pipeline = Pipeline(
        steps=[
            LoadDataFrameStep("df_intermedio.parquet"),
            #GroupByProductStep(),
            DateRelatedFeaturesStep(),

            FilterDatasetByColumn(column="brand", value=brand),
            #Log1pTranformation(),
            #CreateTargetColumDiffStep(target_col="tn"),
            #CreateTargetColumStep(target_col="tn"),
            CreateResidualTargetStep(target_col="tn", window=12),
            ReduceMemoryUsageStep(),

            #ReduceMemoryUsageStep(),
            FeatureEngineeringLagStep(lags=[1,2,3,5,11,23], columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMeanFeatureStep(window=3, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMaxFeatureStep(window=3, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMinFeatureStep(window=3, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMeanFeatureStep(window=9, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMaxFeatureStep(window=9, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMinFeatureStep(window=9, columns=["tn", "cust_request_qty", "stock_final"]),

            ReduceMemoryUsageStep(),

            RollingStdFeatureStep(window=3, columns=["tn", "cust_request_qty"]),
            RollingStdFeatureStep(window=6, columns=["tn", "cust_request_qty"]),
            RollingStdFeatureStep(window=12, columns=["tn", "cust_request_qty"]), 

            RollingSkewFeatureStep(window=3, columns=["tn", "cust_request_qty"]),
            RollingSkewFeatureStep(window=6, columns=["tn", "cust_request_qty"]),
            RollingSkewFeatureStep(window=12, columns=["tn", "cust_request_qty"]),
            ReduceMemoryUsageStep(),

            RollingZscoreFeatureStep(window=3, columns=["tn", "cust_request_qty"]),
            RollingZscoreFeatureStep(window=6, columns=["tn", "cust_request_qty"]),
            RollingZscoreFeatureStep(window=12, columns=["tn", "cust_request_qty"]),
            DiffFeatureStep(periods=1, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=2, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=3, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=4, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=5, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=11, columns=["tn", "cust_request_qty", "stock_final"]),
            FeatureEngineeringProductCatInteractionStep(cat="cat1", tn="tn"),
            FeatureEngineeringProductCatInteractionStep(cat="cat2", tn="tn"),
            FeatureEngineeringProductCatInteractionStep(cat="cat3", tn="tn"),
            #FeatureEngineeringProductCatInteractionStep(cat="product_id", tn="tn"),

            CreateTotalCategoryStep(cat="cat1"),
            CreateTotalCategoryStep(cat="cat2"),
            CreateTotalCategoryStep(cat="cat3"),
            CreateTotalCategoryStep(cat="brand"),
            CreateTotalCategoryStep(cat="customer_id"),
            CreateTotalCategoryStep(cat="product_id"),
                    
            CreateTotalCategoryStep(cat="cat1", tn="stock_final"),
            CreateTotalCategoryStep(cat="cat2", tn="stock_final"),
            CreateTotalCategoryStep(cat="cat3", tn="stock_final"),

            #ReduceMemoryUsageStep(),
            FeatureDivInteractionStep(columns=[
                    ("tn", "tn_cat1_vendidas"), 
                    ("tn", "tn_cat2_vendidas"), 
                    ("tn", "tn_cat3_vendidas"), 
                    ("tn", "tn_brand_vendidas")]
            ),
            #ReduceMemoryUsageStep(),

            FeatureProdInteractionStep(columns=[("tn", "cust_request_qty")]),
            CreateWeightByCustomerStep(),
            CreateWeightByProductStep(),
            ReduceMemoryUsageStep(),
            DeleteBadColumns(),
            FilterProductsIDStep(dfs=["df"]),
            SplitDataFrameStep2(df="df", test_date=29, gap=1),
            PrepareXYStep(),
            TrainModelStep(params={"num_leaves":31, "feature_fraction":0.2}),
            PredictStep(),
            InverseResidualTargetStep(),
            #InverseLog1pTranformation(),
            InverseScalePredictionsStep(),
            IntegratePredictionsStep(),
            EvaluatePredictionsSteps(),
            EvaluatePredictionsOptimizatedSteps(), 
            #PlotFeatureImportanceStep(),

        ],
        optimize_arftifacts_memory=True
    )

    try: 
        pipeline.run()
        
        # Save the results
        pipeline_results[brand] = {
            "eval_df": pipeline.get_artifact("eval_df"),
            "alpha_opt": pipeline.get_artifact("alpha_opt"),
        }
    except Exception as e:
        print(f"Error processing brand {brand}: {e}")
        continue

Processing brand: ARIEL
Executing step: LoadDataFrameStep
Step LoadDataFrameStep completed in 2.33 seconds
Executing step: DateRelatedFeaturesStep
Step DateRelatedFeaturesStep completed in 5.09 seconds
Executing step: FilterDatasetByColumn
(377832, 17)
Step FilterDatasetByColumn completed in 2.74 seconds
Executing step: CreateResidualTargetStep
Step CreateResidualTargetStep completed in 3.45 seconds
Executing step: ReduceMemoryUsageStep
--- Memory usage before: 30.63 MB
--- Memory usage after: 15.13 MB
--- Decreased memory usage by 50.6%

Step ReduceMemoryUsageStep completed in 0.06 seconds
Executing step: FeatureEngineeringLagStep
Step FeatureEngineeringLagStep completed in 0.16 seconds
Executing step: RollingMeanFeatureStep
Step RollingMeanFeatureStep completed in 9.91 seconds
Executing step: RollingMaxFeatureStep
Step RollingMaxFeatureStep completed in 9.55 seconds
Executing step: RollingMinFeatureStep
Step RollingMinFeatureStep completed in 9.74 seconds
Executing step: RollingMeanF

/home/fede/.venvs/labo3/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/home/fede/.venvs/labo3/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


[100]	valid_0's l2: 17.1568	valid_0's total_error: 1.22799
[200]	valid_0's l2: 16.1556	valid_0's total_error: 1.11825
[300]	valid_0's l2: 15.5443	valid_0's total_error: 1.06357
[400]	valid_0's l2: 15.6537	valid_0's total_error: 1.04697
[500]	valid_0's l2: 15.6319	valid_0's total_error: 1.04724
[600]	valid_0's l2: 15.5351	valid_0's total_error: 1.02532
[700]	valid_0's l2: 15.7062	valid_0's total_error: 0.978477
[800]	valid_0's l2: 15.5221	valid_0's total_error: 0.963123
[900]	valid_0's l2: 15.482	valid_0's total_error: 1.00196
[1000]	valid_0's l2: 15.3084	valid_0's total_error: 0.987926
[1100]	valid_0's l2: 15.3274	valid_0's total_error: 0.989223
[1200]	valid_0's l2: 15.2276	valid_0's total_error: 0.994576
[1300]	valid_0's l2: 15.0462	valid_0's total_error: 0.975038
[1400]	valid_0's l2: 15.104	valid_0's total_error: 0.970402
[1500]	valid_0's l2: 15.236	valid_0's total_error: 0.990322
Step TrainModelStep completed in 30.89 seconds
Executing step: PredictStep
Step PredictStep completed in

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


Step InverseResidualTargetStep completed in 1.38 seconds
Executing step: InverseScalePredictionsStep
Step InverseScalePredictionsStep completed in 0.02 seconds
Executing step: IntegratePredictionsStep
Step IntegratePredictionsStep completed in 0.02 seconds
Executing step: EvaluatePredictionsSteps
Error en test: 0.2849

Top 5 productos con mayor error absoluto:
   product_id       target  predictions      tn_real      tn_pred  \
0       20001  1261.322642  1515.850805  1261.322642  1515.850805   
1       20045   115.114431   183.234329   115.114431   183.234329   
7       20649    11.354645    75.591061    11.354645    75.591061   
9       20691    23.896790    56.474383    23.896790    56.474383   
4       20150    68.611140    88.441539    68.611140    88.441539   

   error_absoluto  
0      254.528162  
1       68.119897  
7       64.236416  
9       32.577593  
4       19.830400  
Step EvaluatePredictionsSteps completed in 0.03 seconds
Executing step: EvaluatePredictionsOptimizated

...

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


Step InverseResidualTargetStep completed in 0.75 seconds
Executing step: InverseScalePredictionsStep
Step InverseScalePredictionsStep completed in 0.01 seconds
Executing step: IntegratePredictionsStep
Step IntegratePredictionsStep completed in 0.01 seconds
Executing step: EvaluatePredictionsSteps
Error en test: 0.4291

Top 5 productos con mayor error absoluto:
   product_id    target  predictions   tn_real   tn_pred  error_absoluto
5       21245  0.068272     0.223186  0.068272  0.223186        0.154914
0       21142  0.602781     0.512461  0.602781  0.512461        0.090320
4       21233  0.079804     0.158948  0.079804  0.158948        0.079144
1       21191  0.146995     0.215604  0.146995  0.215604        0.068609
3       21226  0.147976     0.213291  0.147976  0.213291        0.065315
Step EvaluatePredictionsSteps completed in 0.02 seconds
Executing step: EvaluatePredictionsOptimizatedSteps
alfa optimizado: 0.8434
Error en train: 0.2872
Error en test: 0.5489

Top 5 productos con m

/home/fede/.venvs/labo3/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/home/fede/.venvs/labo3/lib/python3.12/site-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


[100]	valid_0's l2: 1.30734e-06	valid_0's total_error: 0.911761
[200]	valid_0's l2: 9.73071e-07	valid_0's total_error: 0.632028
[300]	valid_0's l2: 1.11821e-06	valid_0's total_error: 0.501755
[400]	valid_0's l2: 1.25656e-06	valid_0's total_error: 0.451271
[500]	valid_0's l2: 1.46132e-06	valid_0's total_error: 0.435465
[600]	valid_0's l2: 1.71978e-06	valid_0's total_error: 0.438427
[700]	valid_0's l2: 1.78077e-06	valid_0's total_error: 0.431299
[800]	valid_0's l2: 1.88346e-06	valid_0's total_error: 0.434133
[900]	valid_0's l2: 2.05382e-06	valid_0's total_error: 0.443544
[1000]	valid_0's l2: 2.00365e-06	valid_0's total_error: 0.430082
[1100]	valid_0's l2: 2.03885e-06	valid_0's total_error: 0.41818
[1200]	valid_0's l2: 2.12902e-06	valid_0's total_error: 0.425334
[1300]	valid_0's l2: 2.18616e-06	valid_0's total_error: 0.425328
[1400]	valid_0's l2: 2.19284e-06	valid_0's total_error: 0.423521
[1500]	valid_0's l2: 2.23506e-06	valid_0's total_error: 0.425121
Step TrainModelStep completed in 23

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


In [5]:
pipeline_results

{'ARIEL': {'eval_df':     product_id       target  predictions      tn_real      tn_pred  \
  0        20001  1261.322642  1515.850805  1261.322642  1515.850805   
  1        20045   115.114431   183.234329   115.114431   183.234329   
  2        20102    35.854906    49.655575    35.854906    49.655575   
  3        20135   106.473853   108.751898   106.473853   108.751898   
  4        20150    68.611140    88.441539    68.611140    88.441539   
  5        20378    27.132943    24.909370    27.132943    24.909370   
  6        20611    10.865433    16.318514    10.865433    16.318514   
  7        20649    11.354645    75.591061    11.354645    75.591061   
  8        20682     5.673025    14.260145     5.673025    14.260145   
  9        20691    23.896790    56.474383    23.896790    56.474383   
  10       20919     0.970280     4.297042     0.970280     4.297042   
  
      error_absoluto  
  0       254.528162  
  1        68.119897  
  2        13.800669  
  3         2.278045 

In [6]:
final_eval_scaled = pd.DataFrame()
for brand, result in pipeline_results.items():
    alpha_opt = result["alpha_opt"]
    eval_df = result["eval_df"]
    eval_df["tn_pred_scaled"] = eval_df["tn_pred"] * alpha_opt
    final_eval_scaled = pd.concat([final_eval_scaled, eval_df], ignore_index=True)


In [7]:
final_eval_scaled

,product_id,target,predictions,tn_real,tn_pred,error_absoluto,tn_pred_scaled
0,20001,1261.322642,1515.850805,1261.322642,1515.850805,254.528162,1276.466155
1,20045,115.114431,183.234329,115.114431,183.234329,68.119897,154.297783
2,20102,35.854906,49.655575,35.854906,49.655575,13.800669,41.813918
3,20135,106.473853,108.751898,106.473853,108.751898,2.278045,91.577691
4,20150,68.611140,88.441539,68.611140,88.441539,19.830400,74.474764
...,...,...,...,...,...,...,...
729,21248,0.021150,-0.006976,0.021150,-0.006976,0.028126,-0.000000
730,21256,0.029625,-0.014238,0.029625,-0.014238,0.043863,-0.000000
731,21259,0.031038,-0.013992,0.031038,-0.013992,0.045030,-0.000000
732,21262,0.022550,-0.012560,0.022550,-0.012560,0.035110,-0.000000


In [8]:
total_error_1 = np.sum(np.abs(final_eval_scaled["tn_pred"] - final_eval_scaled["tn_real"])) / np.sum(final_eval_scaled["tn_real"])
total_error_2 = np.sum(np.abs(final_eval_scaled["tn_pred_scaled"] - final_eval_scaled["tn_real"])) / np.sum(final_eval_scaled["tn_real"])
print(f"Total error without scaling: {total_error_1}")
print(f"Total error with scaling: {total_error_2}")

Total error without scaling: 0.37683290545212056
Total error with scaling: 0.21524080665513978


In [9]:
# entreno pipeline final para kaggle
kaggle_results = {}
for brand, result in pipeline_results.items():
    print(f"Processing brand: {brand}")
    alpha_opt = result["alpha_opt"]
    pipeline = Pipeline(
        steps=[
            LoadDataFrameStep("df_intermedio.parquet"),
            GroupByProductStep(),
            DateRelatedFeaturesStep(),

            FilterDatasetByColumn(column="brand", value=brand),
            #Log1pTranformation(),
            #CreateTargetColumDiffStep(target_col="tn"),
            #CreateTargetColumStep(target_col="tn"),
            CreateResidualTargetStep(target_col="tn", window=12),
            ReduceMemoryUsageStep(),

            #ReduceMemoryUsageStep(),
            FeatureEngineeringLagStep(lags=[1,2,3,5,11,23], columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMeanFeatureStep(window=3, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMaxFeatureStep(window=3, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMinFeatureStep(window=3, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMeanFeatureStep(window=9, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMaxFeatureStep(window=9, columns=["tn", "cust_request_qty", "stock_final"]),
            RollingMinFeatureStep(window=9, columns=["tn", "cust_request_qty", "stock_final"]),

            ReduceMemoryUsageStep(),

            RollingStdFeatureStep(window=3, columns=["tn", "cust_request_qty"]),
            RollingStdFeatureStep(window=6, columns=["tn", "cust_request_qty"]),
            RollingStdFeatureStep(window=12, columns=["tn", "cust_request_qty"]), 

            RollingSkewFeatureStep(window=3, columns=["tn", "cust_request_qty"]),
            RollingSkewFeatureStep(window=6, columns=["tn", "cust_request_qty"]),
            RollingSkewFeatureStep(window=12, columns=["tn", "cust_request_qty"]),
            ReduceMemoryUsageStep(),

            RollingZscoreFeatureStep(window=3, columns=["tn", "cust_request_qty"]),
            RollingZscoreFeatureStep(window=6, columns=["tn", "cust_request_qty"]),
            RollingZscoreFeatureStep(window=12, columns=["tn", "cust_request_qty"]),
            DiffFeatureStep(periods=1, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=2, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=3, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=4, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=5, columns=["tn", "cust_request_qty", "stock_final"]),
            DiffFeatureStep(periods=11, columns=["tn", "cust_request_qty", "stock_final"]),
            FeatureEngineeringProductCatInteractionStep(cat="cat1", tn="tn"),
            FeatureEngineeringProductCatInteractionStep(cat="cat2", tn="tn"),
            FeatureEngineeringProductCatInteractionStep(cat="cat3", tn="tn"),
            #FeatureEngineeringProductCatInteractionStep(cat="product_id", tn="tn"),

            CreateTotalCategoryStep(cat="cat1"),
            CreateTotalCategoryStep(cat="cat2"),
            CreateTotalCategoryStep(cat="cat3"),
            CreateTotalCategoryStep(cat="brand"),
            CreateTotalCategoryStep(cat="customer_id"),
            CreateTotalCategoryStep(cat="product_id"),
                    
            CreateTotalCategoryStep(cat="cat1", tn="stock_final"),
            CreateTotalCategoryStep(cat="cat2", tn="stock_final"),
            CreateTotalCategoryStep(cat="cat3", tn="stock_final"),

            #ReduceMemoryUsageStep(),
            FeatureDivInteractionStep(columns=[
                    ("tn", "tn_cat1_vendidas"), 
                    ("tn", "tn_cat2_vendidas"), 
                    ("tn", "tn_cat3_vendidas"), 
                    ("tn", "tn_brand_vendidas")]
            ),
            #ReduceMemoryUsageStep(),

            FeatureProdInteractionStep(columns=[("tn", "cust_request_qty")]),
            CreateWeightByCustomerStep(),
            CreateWeightByProductStep(),
            ReduceMemoryUsageStep(),
            DeleteBadColumns(),
            FilterProductsIDStep(dfs=["df"]),
            SplitDataFrameStep2(df="df", test_date=35, gap=1),
            PrepareXYStep(),
            TrainModelStep(params={"num_leaves":31, "feature_fraction":0.2}),
            PredictStep(),
            InverseResidualTargetStep(),
            #InverseLog1pTranformation(),
            InverseScalePredictionsStep(),
            IntegratePredictionsStep(),
            #PlotFeatureImportanceStep(),
            KaggleSubmissionStep(),
            KaggleSubmissionStep(alpha_opt=alpha_opt, experiment="alpha"),

        ],
        optimize_arftifacts_memory=True
    )

    try: 
        pipeline.run()
        
        # Save the results
        kaggle_results[brand] = {
            "submission": pipeline.get_artifact("submission"),
            "submission_alpha": pipeline.get_artifact("submissionalpha"),
        }
    except Exception as e:
        print(f"Error processing brand {brand}: {e}")
        continue

Processing brand: ARIEL
Executing step: LoadDataFrameStep
Step LoadDataFrameStep completed in 2.12 seconds
Executing step: GroupByProductStep
Step GroupByProductStep completed in 1.46 seconds
Executing step: DateRelatedFeaturesStep
Step DateRelatedFeaturesStep completed in 0.04 seconds
Executing step: FilterDatasetByColumn
(761, 16)
Step FilterDatasetByColumn completed in 0.13 seconds
Executing step: CreateResidualTargetStep
Step CreateResidualTargetStep completed in 0.01 seconds
Executing step: ReduceMemoryUsageStep
--- Memory usage before: 0.06 MB
--- Memory usage after: 0.03 MB
--- Decreased memory usage by 52.0%

Step ReduceMemoryUsageStep completed in 0.01 seconds
Executing step: FeatureEngineeringLagStep
Step FeatureEngineeringLagStep completed in 0.01 seconds
Executing step: RollingMeanFeatureStep
Step RollingMeanFeatureStep completed in 0.02 seconds
Executing step: RollingMaxFeatureStep
Step RollingMaxFeatureStep completed in 0.02 seconds
Executing step: RollingMinFeatureStep
S

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


Step LoadDataFrameStep completed in 2.00 seconds
Executing step: GroupByProductStep
Step GroupByProductStep completed in 1.45 seconds
Executing step: DateRelatedFeaturesStep
Step DateRelatedFeaturesStep completed in 0.04 seconds
Executing step: FilterDatasetByColumn
(1843, 16)
Step FilterDatasetByColumn completed in 0.01 seconds
Executing step: CreateResidualTargetStep
Step CreateResidualTargetStep completed in 0.02 seconds
Executing step: ReduceMemoryUsageStep
--- Memory usage before: 0.15 MB
--- Memory usage after: 0.07 MB
--- Decreased memory usage by 52.5%

Step ReduceMemoryUsageStep completed in 0.01 seconds
Executing step: FeatureEngineeringLagStep
Step FeatureEngineeringLagStep completed in 0.01 seconds
Executing step: RollingMeanFeatureStep
Step RollingMeanFeatureStep completed in 0.03 seconds
Executing step: RollingMaxFeatureStep
Step RollingMaxFeatureStep completed in 0.03 seconds
Executing step: RollingMinFeatureStep
Step RollingMinFeatureStep completed in 0.03 seconds
Execu

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


Step InverseResidualTargetStep completed in 0.03 seconds
Executing step: InverseScalePredictionsStep
Step InverseScalePredictionsStep completed in 0.00 seconds
Executing step: IntegratePredictionsStep
Step IntegratePredictionsStep completed in 0.00 seconds
Executing step: KaggleSubmissionStep
Step KaggleSubmissionStep completed in 0.01 seconds
Executing step: KaggleSubmissionStep
Step KaggleSubmissionStep completed in 0.00 seconds
Processing brand: NATURA
Executing step: LoadDataFrameStep
Step LoadDataFrameStep completed in 2.03 seconds
Executing step: GroupByProductStep
Step GroupByProductStep completed in 1.48 seconds
Executing step: DateRelatedFeaturesStep
Step DateRelatedFeaturesStep completed in 0.04 seconds
Executing step: FilterDatasetByColumn
(1281, 16)
Step FilterDatasetByColumn completed in 0.01 seconds
Executing step: CreateResidualTargetStep
Step CreateResidualTargetStep completed in 0.01 seconds
Executing step: ReduceMemoryUsageStep
--- Memory usage before: 0.10 MB
--- Mem

...

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


Step LoadDataFrameStep completed in 1.91 seconds
Executing step: GroupByProductStep
Step GroupByProductStep completed in 1.53 seconds
Executing step: DateRelatedFeaturesStep
Step DateRelatedFeaturesStep completed in 0.04 seconds
Executing step: FilterDatasetByColumn
(112, 16)
Step FilterDatasetByColumn completed in 0.01 seconds
Executing step: CreateResidualTargetStep
Step CreateResidualTargetStep completed in 0.01 seconds
Executing step: ReduceMemoryUsageStep
--- Memory usage before: 0.01 MB
--- Memory usage after: 0.01 MB
--- Decreased memory usage by 48.1%

Step ReduceMemoryUsageStep completed in 0.01 seconds
Executing step: FeatureEngineeringLagStep
Step FeatureEngineeringLagStep completed in 0.01 seconds
Executing step: RollingMeanFeatureStep
Step RollingMeanFeatureStep completed in 0.01 seconds
Executing step: RollingMaxFeatureStep
Step RollingMaxFeatureStep completed in 0.01 seconds
Executing step: RollingMinFeatureStep
Step RollingMinFeatureStep completed in 0.01 seconds
Execut

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


Step InverseResidualTargetStep completed in 0.02 seconds
Executing step: InverseScalePredictionsStep
Step InverseScalePredictionsStep completed in 0.00 seconds
Executing step: IntegratePredictionsStep
Step IntegratePredictionsStep completed in 0.00 seconds
Executing step: KaggleSubmissionStep
Step KaggleSubmissionStep completed in 0.00 seconds
Executing step: KaggleSubmissionStep
Step KaggleSubmissionStep completed in 0.00 seconds
Processing brand: VICHY
Executing step: LoadDataFrameStep
Step LoadDataFrameStep completed in 1.91 seconds
Executing step: GroupByProductStep
Step GroupByProductStep completed in 1.54 seconds
Executing step: DateRelatedFeaturesStep
Step DateRelatedFeaturesStep completed in 0.04 seconds
Executing step: FilterDatasetByColumn
(119, 16)
Step FilterDatasetByColumn completed in 0.01 seconds
Executing step: CreateResidualTargetStep
Step CreateResidualTargetStep completed in 0.01 seconds
Executing step: ReduceMemoryUsageStep
--- Memory usage before: 0.01 MB
--- Memor

/tmp/ipykernel_98974/3669324445.py:148: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['base_prediction'] = (


In [10]:
final_submission_scaled = pd.DataFrame(columns=["product_id", "tn"])
final_submission_no_scaled = pd.DataFrame(columns=["product_id", "tn"])
for brand, result in kaggle_results.items():
    submission_normal = result["submission"]
    submission_alpha = result["submission_alpha"]
    final_submission_no_scaled = pd.concat([final_submission_no_scaled, submission_normal], ignore_index=True)
    final_submission_scaled = pd.concat([final_submission_scaled, submission_alpha], ignore_index=True)



In [11]:
final_submission_no_scaled

,product_id,tn
0,20001,1345.738387
1,20045,170.647413
2,20102,11.787400
3,20135,137.287058
4,20150,38.643878
...,...,...
775,21248,-0.087762
776,21256,-0.097195
777,21259,-0.083637
778,21262,-0.096032


In [12]:
final_submission_scaled

,product_id,tn
0,20001,1133.218058
1,20045,143.698605
2,20102,9.925922
3,20135,115.606551
4,20150,32.541199
...,...,...
775,21248,0.000000
776,21256,0.000000
777,21259,0.000000
778,21262,0.000000


In [14]:
# pasar final_submission_no_scaled a menor a 0 a 0
final_submission_no_scaled["tn"] = final_submission_no_scaled["tn"].apply(lambda x: max(x, 0))


In [15]:
final_submission_scaled["tn"] = final_submission_scaled["tn"].apply(lambda x: max(x, 0))

In [16]:
final_submission_no_scaled.to_csv("submission_no_scaled.csv", index=False)
final_submission_scaled.to_csv("submission_scaled.csv", index=False)